In [5]:
from keras.datasets import fashion_mnist,mnist
import wandb
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import copy
import argparse
from types import SimpleNamespace


from utilities.NeuralNetwork import NN
from utilities.HelperFunctions import OneHotEncoder
from utilities.config import * # reading global variables 


def pre_process(x):
    '''
    reshape and normalized the data to bring to 0-1 scale.
    '''
    x=x.reshape(-1,784)
    x=x/255
    return x

def load_data(dataset=fmnist_dataset,split_size=valid_split_size):
    
    '''
    loads and returns data after doing train-valid split.
    '''
    if dataset==fmnist_dataset:
        (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
    elif dataset==mnist_dataset:
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
    
    
    x_train,x_valid=x_train[:int(len(x_train)*split_size)],x_train[int(len(x_train)*split_size):] #splitting train into train and valid
    y_train,y_valid=y_train[:int(len(y_train)*split_size)],y_train[int(len(y_train)*split_size):]

    x_train=pre_process(x_train)
    x_valid=pre_process(x_valid)
    x_test=pre_process(x_test) 
    
    one_hot=OneHotEncoder(10)
    y_train=one_hot.transform(y_train)
    y_valid=one_hot.transform(y_valid)
    y_test=one_hot.transform(y_test)
    
    
    return x_train,y_train,x_valid,y_valid,x_test,y_test

x_train,y_train,x_valid,y_valid,x_test,y_test=load_data(fmnist_dataset,1-valid_split_size)
    

In [6]:
sweep_config = {
    'method': 'random', #grid, random,#bayes
    'name' : 'rand_sweep_cross_entropy',
    'metric': {
      'name': 'valid accuracy',
      'goal': 'maximize'  
    },
    'parameters': {
        'epochs': {
            'values': [1]
        },
        'num_layers':{
            'values':[3,4,5]
        },
         'hidden_size':{
            'values':[32,64,128]
        },
        'weight_decay':{
            'values':[0,0.0005,0.5]
        },
         'learning_rate':{
            'values':[0.001,0.0001]
        },
         'optimizer': {
            'values': ['sgd','momentum','nag','rmsprop','adam','nadam']           
        },
        'batch_size': {
            'values': [16,32,64]
        },
         'weight_init':{
            'values':['normal','xavier']            
        },
        'activation': {
            'values': ['sigmoid','tanh','relu']
        }, 
        'loss': {
            'values': [entropy_loss]
        }, 
        
        
    }
}

sweep_id = wandb.sweep(sweep=sweep_config, project='temp')

Create sweep with ID: 8njxnfqh
Sweep URL: https://wandb.ai/cs22m080/temp/sweeps/8njxnfqh


In [7]:
def main():
    '''
    WandB calls main function each time with differnet combination.
    
    We can retrive the same and use the same values for our hypermeters.
    
    '''
    with wandb.init() as run:
        params={}
        params=dict(wandb.config)
        params=SimpleNamespace(**params)
        
        layers=[params.hidden_size]*params.num_layers
        run_name="opt_"+wandb.config.optimizer+"-ac_"+wandb.config.activation+"-batch_"+str(wandb.config.batch_size)\
                + "-nlayers_"+str(wandb.config.num_layers)+"-lr_"+str(wandb.config.learning_rate)
        wandb.run.name=run_name
        layers.append(10)
        obj=NN(784,layers,params)
        
        train_losses,tr_ac,valid_losses,val_ac=obj.train(x_train.T,y_train,x_valid.T,y_valid)
    
wandb.agent(sweep_id, function=main,count=5)
wandb.finish()


wandb: Agent Starting Run: stnlnp10 with config:
wandb: 	activation: relu
wandb: 	batch_size: 64
wandb: 	epochs: 1
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.0001
wandb: 	loss: cross_entropy_loss
wandb: 	num_layers: 5
wandb: 	optimizer: adam
wandb: 	weight_decay: 0
wandb: 	weight_init: normal
wandb: Currently logged in as: cs22m080. Use `wandb login --relogin` to force relogin


epoch 1 : train loss = 0.87 valid loss = 0.53 train accuracy = 81.14 valid accuracy = 81.02


train accuracy,▁
train loss,▁
valid accuracy,▁
valid loss,▁
train accuracy,81.13889
train loss,0.87325
valid accuracy,81.01667
valid loss,0.53335


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: nzi80p42 with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 32
wandb: 	epochs: 1
wandb: 	hidden_size: 64
wandb: 	learning_rate: 0.0001
wandb: 	loss: cross_entropy_loss
wandb: 	num_layers: 3
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0.0005
wandb: 	weight_init: normal


epoch 1 : train loss = 1.88 valid loss = 1.29 train accuracy = 59.91 valid accuracy = 60.73


...

epoch 1 : train loss = 0.51 valid loss = 0.41 train accuracy = 86.23 valid accuracy = 85.52


train accuracy,▁
train loss,▁
valid accuracy,▁
valid loss,▁
train accuracy,86.22593
train loss,0.51481
valid accuracy,85.51667
valid loss,0.40733


wandb: Agent Starting Run: tn9v9s9h with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 16
wandb: 	epochs: 1
wandb: 	hidden_size: 128
wandb: 	learning_rate: 0.001
wandb: 	loss: cross_entropy_loss
wandb: 	num_layers: 4
wandb: 	optimizer: rmsprop
wandb: 	weight_decay: 0.0005
wandb: 	weight_init: xavier


epoch 1 : train loss = 0.74 valid loss = 0.51 train accuracy = 82.97 valid accuracy = 81.97


train accuracy,▁
train loss,▁
valid accuracy,▁
valid loss,▁
train accuracy,82.96852
train loss,0.74186
valid accuracy,81.96667
valid loss,0.50887
